# SQL analytics model — Climate extremes, inflation (IPC) and real wages (RIPTE) in Argentina

This notebook builds an **end-to-end analytical dataset in DuckDB** integrating:
- **INDEC IPC (consumer price index)** by category (COICOP) and region,
- **RIPTE** (average registered wages, total country),
- a national monthly **climate proxy** (from `02_climate_processing.ipynb`).

It then answers:
1. How do **climate extremes** relate to **inflation** (general vs food)?
2. Does the relationship persist with a **1‑month lag**?
3. Are there **regional differences** in food inflation vs the national average?

**Database output**
- `data/processed/analytics.duckdb`

**Required input files**
- `data/raw/serie_ipc_aperturas.csv` (INDEC)
- `data/raw/ripte.csv` (official RIPTE monthly series)
- `data/processed/climate_monthly_avg.csv` (generated by Notebook 02)


## 0. Setup (DuckDB connection)

In [1]:
!pip install duckdb


     -------------------------------------- 12.3/12.3 MB 597.3 kB/s eta 0:00:00


In [15]:
import duckdb
con = duckdb.connect("../data/processed/analytics.duckdb")


## 1. Stage INDEC IPC raw file

We load the INDEC IPC file exactly as published.  
Key handling:
- delimiter `;`
- `latin-1` encoding
- header is not reliable, so we import as generic `column0..column7` and clean later.


In [26]:
con.execute("""
CREATE OR REPLACE TABLE stg_ipc_raw AS
SELECT *
FROM read_csv(
  '../data/raw/serie_ipc_aperturas.csv',
  delim=';',
  header=false,
  encoding='latin-1'
);
""")


In [28]:
con.execute("SELECT * FROM stg_ipc_raw LIMIT 15").df()



,column0,column1,column2,column3,column4,column5,column6,column7
0,Codigo,Descripcion_aperturas,Clasificador,Periodo,Indice_IPC,v_m_IPC,v_i_a_IPC,Region
1,0,Nivel general,Nivel general y divisiones COICOP,201612,100,NA,NA,Pampeana
2,0,Nivel general,Nivel general y divisiones COICOP,201612,100,NA,NA,Noreste
3,0,Nivel general,Nivel general y divisiones COICOP,201612,100,NA,NA,Noroeste
4,0,Nivel general,Nivel general y divisiones COICOP,201612,100,NA,NA,Cuyo
5,0,Nivel general,Nivel general y divisiones COICOP,201612,100,NA,NA,Patagonia
6,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100,NA,NA,Pampeana
7,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100,NA,NA,Noreste
8,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100,NA,NA,Noroeste
9,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100,NA,NA,Cuyo


## 2. Clean IPC and create typed staging table

Cleaning steps:
- cast columns to meaningful names and types
- convert decimal commas to dots for numeric fields
- convert `NA` to NULL
- drop the header row embedded in the file


In [32]:
con.execute("""
CREATE OR REPLACE TABLE stg_ipc_clean AS
SELECT
  CAST(column0 AS VARCHAR) AS codigo,
  CAST(column1 AS VARCHAR) AS descripcion_aperturas,
  CAST(column2 AS VARCHAR) AS clasificador,
  CAST(column3 AS INTEGER) AS periodo_yyyymm,

  -- índice IPC (coma decimal → punto)
  NULLIF(
    REPLACE(CAST(column4 AS VARCHAR), ',', '.'),
    'NA'
  )::DOUBLE AS indice_ipc,

  -- variación mensual (%)
  NULLIF(
    REPLACE(CAST(column5 AS VARCHAR), ',', '.'),
    'NA'
  )::DOUBLE AS vm_ipc,

  -- variación interanual (%)
  NULLIF(
    REPLACE(CAST(column6 AS VARCHAR), ',', '.'),
    'NA'
  )::DOUBLE AS via_ipc,

  CAST(column7 AS VARCHAR) AS region
FROM stg_ipc_raw
WHERE column0 <> 'Codigo';
""")



In [33]:
con.execute("SELECT * FROM stg_ipc_clean LIMIT 10").df()


,codigo,descripcion_aperturas,clasificador,periodo_yyyymm,indice_ipc,vm_ipc,via_ipc,region
0,0,Nivel general,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Pampeana
1,0,Nivel general,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Noreste
2,0,Nivel general,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Noroeste
3,0,Nivel general,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Cuyo
4,0,Nivel general,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Patagonia
5,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Pampeana
6,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Noreste
7,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Noroeste
8,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Cuyo
9,01,Alimentos y bebidas no alcohólicas,Nivel general y divisiones COICOP,201612,100.0,NaN,NaN,Patagonia


## 3. Dimensional model (calendar dimension + IPC fact tables)

In [34]:
con.execute("""
CREATE OR REPLACE TABLE dim_month AS
WITH months AS (
  SELECT DATE_TRUNC('month', d)::DATE AS month_date
  FROM generate_series(DATE '2016-12-01', DATE '2024-12-01', INTERVAL 1 month) t(d)
)
SELECT
  month_date,
  STRFTIME(month_date, '%Y%m')::INTEGER AS periodo_yyyymm,
  STRFTIME(month_date, '%Y-%m') AS year_month,
  EXTRACT(year FROM month_date)::INTEGER AS year,
  EXTRACT(month FROM month_date)::INTEGER AS month,
  EXTRACT(quarter FROM month_date)::INTEGER AS quarter
FROM months;
""")


Create IPC fact tables at **region-month** and **national-month** grains.

In [36]:
con.execute("""
CREATE OR REPLACE TABLE fact_ipc_region_monthly AS
SELECT
  m.month_date,
  m.year_month,
  c.periodo_yyyymm,
  c.region,
  c.codigo,
  c.descripcion_aperturas,
  c.indice_ipc,
  c.vm_ipc,
  c.via_ipc
FROM stg_ipc_clean c
JOIN dim_month m
  ON c.periodo_yyyymm = m.periodo_yyyymm;
""")


In [37]:
con.execute("""
CREATE OR REPLACE TABLE fact_ipc_national_monthly AS
SELECT
  month_date,
  year_month,
  codigo,
  descripcion_aperturas,
  AVG(indice_ipc) AS indice_ipc,
  AVG(vm_ipc) AS vm_ipc,
  AVG(via_ipc) AS via_ipc
FROM fact_ipc_region_monthly
GROUP BY 1,2,3,4;
""")


## 4. Load and model climate monthly dataset

This uses the CSV exported by `02_climate_processing.ipynb`.

In [38]:
con.execute("""
CREATE OR REPLACE TABLE stg_climate_monthly AS
SELECT * FROM read_csv_auto('../data/processed/climate_monthly_avg.csv');
""")


In [39]:
con.execute("SELECT * FROM stg_climate_monthly LIMIT 5").df()


,year_month,tavg,tmin,tmax
0,2017-01,26.184946,20.795860,32.185108
1,2017-02,24.659286,19.932202,30.517798
2,2017-03,21.813925,16.893333,27.658602
3,2017-04,17.732722,13.462611,24.071000
4,2017-05,14.765860,10.345538,20.486452


In [40]:
con.execute("""
CREATE OR REPLACE TABLE fact_climate_monthly AS
SELECT
  m.month_date,
  m.year_month,
  CAST(c.tavg AS DOUBLE) AS tavg,
  CAST(c.tmin AS DOUBLE) AS tmin,
  CAST(c.tmax AS DOUBLE) AS tmax
FROM stg_climate_monthly c
JOIN dim_month m
  ON c.year_month = m.year_month;
""")


## 5. Load and model RIPTE (monthly wages)

The RIPTE CSV already comes typed (DATE + DOUBLE), so we only:
- standardize to month start (`month_date`)
- create a `year_month` join key


In [42]:
con.execute("""
CREATE OR REPLACE TABLE stg_ripte_raw AS
SELECT *
FROM read_csv_auto('../data/raw/ripte.csv');
""")



In [47]:
con.execute("SELECT * FROM stg_ripte_raw LIMIT 10").df()
con.execute("DESCRIBE stg_ripte_raw").df()


,column_name,column_type,null,key,default,extra
0,indice_tiempo,DATE,YES,None,None,None
1,ripte,DOUBLE,YES,None,None,None


In [49]:
con.execute("""
CREATE OR REPLACE TABLE fact_ripte_monthly AS
SELECT
  DATE_TRUNC('month', indice_tiempo)::DATE AS month_date,
  STRFTIME(DATE_TRUNC('month', indice_tiempo)::DATE, '%Y-%m') AS year_month,
  ripte AS ripte_nominal
FROM stg_ripte_raw;
""")



In [50]:
con.execute("SELECT * FROM fact_ripte_monthly ORDER BY month_date LIMIT 10").df()



,month_date,year_month,ripte_nominal
0,1994-07-01,1994-07,874.87
1,1994-08-01,1994-08,893.00
2,1994-09-01,1994-09,907.99
3,1994-10-01,1994-10,910.75
4,1994-11-01,1994-11,916.93
5,1994-12-01,1994-12,936.83
6,1995-01-01,1995-01,934.85
7,1995-02-01,1995-02,928.29
8,1995-03-01,1995-03,931.37
9,1995-04-01,1995-04,909.07


## 6. Build analytical views (IPC key series, real wages, climate regimes)

### 6.1 Key IPC series (Nivel general vs Alimentos)

In [61]:
con.execute("""
CREATE OR REPLACE VIEW vw_ipc_keyseries AS
SELECT
  month_date,
  year_month,
  MAX(CASE WHEN descripcion_aperturas = 'Nivel general' THEN indice_ipc END) AS ipc_ng_index,
  MAX(CASE WHEN descripcion_aperturas = 'Nivel general' THEN vm_ipc END)     AS ipc_ng_mom,
  MAX(CASE WHEN descripcion_aperturas = 'Alimentos y bebidas no alcohólicas' THEN indice_ipc END) AS ipc_food_index,
  MAX(CASE WHEN descripcion_aperturas = 'Alimentos y bebidas no alcohólicas' THEN vm_ipc END)     AS ipc_food_mom
FROM fact_ipc_national_monthly
GROUP BY 1,2;
""")


In [64]:
con.execute("SELECT * FROM vw_monthly_core ORDER BY month_date LIMIT 12").df()


,month_date,year_month,tavg,tmin,tmax,ipc_ng_index,ipc_ng_mom,ipc_food_index,ipc_food_mom
0,2017-01-01,2017-01,26.184946,20.795860,32.185108,101.773533,1.783333,101.298300,1.283333
1,2017-02-01,2017-02,24.659286,19.932202,30.517798,103.676800,1.883333,103.171283,1.833333
2,2017-03-01,2017-03,21.813925,16.893333,27.658602,106.260583,2.483333,105.480800,2.233333
3,2017-04-01,2017-04,17.732722,13.462611,24.071000,109.027450,2.583333,107.882067,2.266667
4,2017-05-01,2017-05,14.765860,10.345538,20.486452,110.771517,1.600000,109.527417,1.516667
5,2017-06-01,2017-06,12.081444,6.519889,19.253667,112.027950,1.133333,110.367833,0.766667
6,2017-07-01,2017-07,12.011935,7.405000,18.618656,113.759150,1.550000,111.476950,1.000000
7,2017-08-01,2017-08,13.981452,8.401935,20.508441,115.330667,1.400000,113.517033,1.833333
8,2017-09-01,2017-09,15.324889,10.559111,21.558333,117.263467,1.666667,115.650867,1.866667
9,2017-10-01,2017-10,18.318656,12.384301,25.105484,119.161767,1.616667,117.516033,1.616667


### 6.2 Core monthly panel (climate + IPC) and climate extreme labeling

In [63]:
con.execute("""
CREATE OR REPLACE VIEW vw_monthly_core AS
SELECT
  m.month_date,
  m.year_month,
  c.tavg,
  c.tmin,
  c.tmax,
  i.ipc_ng_index,
  i.ipc_ng_mom,
  i.ipc_food_index,
  i.ipc_food_mom
FROM dim_month m
LEFT JOIN fact_climate_monthly c USING (month_date, year_month)
LEFT JOIN vw_ipc_keyseries i USING (month_date, year_month)
WHERE m.month_date BETWEEN DATE '2017-01-01' AND DATE '2024-12-01';
""")


In [64]:
con.execute("SELECT * FROM vw_monthly_core ORDER BY month_date LIMIT 12").df()


,month_date,year_month,tavg,tmin,tmax,ipc_ng_index,ipc_ng_mom,ipc_food_index,ipc_food_mom
0,2017-01-01,2017-01,26.184946,20.795860,32.185108,101.773533,1.783333,101.298300,1.283333
1,2017-02-01,2017-02,24.659286,19.932202,30.517798,103.676800,1.883333,103.171283,1.833333
2,2017-03-01,2017-03,21.813925,16.893333,27.658602,106.260583,2.483333,105.480800,2.233333
3,2017-04-01,2017-04,17.732722,13.462611,24.071000,109.027450,2.583333,107.882067,2.266667
4,2017-05-01,2017-05,14.765860,10.345538,20.486452,110.771517,1.600000,109.527417,1.516667
5,2017-06-01,2017-06,12.081444,6.519889,19.253667,112.027950,1.133333,110.367833,0.766667
6,2017-07-01,2017-07,12.011935,7.405000,18.618656,113.759150,1.550000,111.476950,1.000000
7,2017-08-01,2017-08,13.981452,8.401935,20.508441,115.330667,1.400000,113.517033,1.833333
8,2017-09-01,2017-09,15.324889,10.559111,21.558333,117.263467,1.666667,115.650867,1.866667
9,2017-10-01,2017-10,18.318656,12.384301,25.105484,119.161767,1.616667,117.516033,1.616667


In [65]:
con.execute("""
CREATE OR REPLACE VIEW vw_core_with_extremes AS
WITH stats AS (
  SELECT
    AVG(tavg) AS mu,
    STDDEV_SAMP(tavg) AS sigma
  FROM vw_monthly_core
  WHERE tavg IS NOT NULL
),
z AS (
  SELECT
    v.*,
    (v.tavg - s.mu) / NULLIF(s.sigma, 0) AS z_tavg
  FROM vw_monthly_core v
  CROSS JOIN stats s
)
SELECT
  *,
  CASE
    WHEN z_tavg >= 1.5 THEN 'heat_extreme'
    WHEN z_tavg <= -1.5 THEN 'cold_extreme'
    ELSE 'normal'
  END AS climate_regime
FROM z;
""")


In [66]:
con.execute("""
SELECT year_month, tavg, z_tavg, climate_regime, ipc_ng_mom, ipc_food_mom
FROM vw_core_with_extremes
ORDER BY month_date
LIMIT 15;
""").df()


,year_month,tavg,z_tavg,climate_regime,ipc_ng_mom,ipc_food_mom
0,2017-01,26.184946,1.496477,normal,1.783333,1.283333
1,2017-02,24.659286,1.200534,normal,1.883333,1.833333
2,2017-03,21.813925,0.648600,normal,2.483333,2.233333
3,2017-04,17.732722,-0.143059,normal,2.583333,2.266667
4,2017-05,14.765860,-0.718561,normal,1.600000,1.516667
5,2017-06,12.081444,-1.239276,normal,1.133333,0.766667
6,2017-07,12.011935,-1.252759,normal,1.550000,1.000000
7,2017-08,13.981452,-0.870718,normal,1.400000,1.833333
8,2017-09,15.324889,-0.610123,normal,1.666667,1.866667
9,2017-10,18.318656,-0.029401,normal,1.616667,1.616667


### 6.3 Real wage index (RIPTE deflated by IPC Nivel general)

In [51]:
con.execute("""
CREATE OR REPLACE VIEW vw_real_wage AS
WITH ipc_ng AS (
  SELECT
    month_date,
    year_month,
    indice_ipc AS ipc_ng_index
  FROM fact_ipc_national_monthly
  WHERE descripcion_aperturas = 'Nivel general'
)
SELECT
  r.month_date,
  r.year_month,
  r.ripte_nominal,
  i.ipc_ng_index,
  (r.ripte_nominal / i.ipc_ng_index) * 100.0 AS real_wage_index
FROM fact_ripte_monthly r
JOIN ipc_ng i
  USING (month_date, year_month);
""")


In [52]:
con.execute("SELECT * FROM vw_real_wage ORDER BY month_date LIMIT 10").df()


,month_date,year_month,ripte_nominal,ipc_ng_index,real_wage_index
0,2016-12-01,2016-12,20690.14,100.000000,20690.140000
1,2017-01-01,2017-01,21048.21,101.773533,20681.418155
2,2017-02-01,2017-02,21483.03,103.676800,20721.154588
3,2017-03-01,2017-03,22285.48,106.260583,20972.480388
4,2017-04-01,2017-04,22650.53,109.027450,20775.070865
5,2017-05-01,2017-05,23029.98,110.771517,20790.525121
6,2017-06-01,2017-06,23469.98,112.027950,20950.111111
7,2017-07-01,2017-07,24489.17,113.759150,21527.209020
8,2017-08-01,2017-08,24700.42,115.330667,21417.044325
9,2017-09-01,2017-09,25136.35,117.263467,21435.789607


In [53]:
con.execute("""
CREATE OR REPLACE VIEW vw_real_wage_base100 AS
WITH base AS (
  SELECT
    *,
    FIRST_VALUE(real_wage_index) OVER (ORDER BY month_date) AS base_val
  FROM vw_real_wage
)
SELECT
  month_date,
  year_month,
  ripte_nominal,
  ipc_ng_index,
  real_wage_index,
  100.0 * real_wage_index / NULLIF(base_val, 0) AS real_wage_index_base100
FROM base;
""")


In [54]:
con.execute("SELECT * FROM vw_real_wage_base100 ORDER BY month_date LIMIT 10").df()


,month_date,year_month,ripte_nominal,ipc_ng_index,real_wage_index,real_wage_index_base100
0,2016-12-01,2016-12,20690.14,100.000000,20690.140000,100.000000
1,2017-01-01,2017-01,21048.21,101.773533,20681.418155,99.957845
2,2017-02-01,2017-02,21483.03,103.676800,20721.154588,100.149900
3,2017-03-01,2017-03,22285.48,106.260583,20972.480388,101.364613
4,2017-04-01,2017-04,22650.53,109.027450,20775.070865,100.410490
5,2017-05-01,2017-05,23029.98,110.771517,20790.525121,100.485183
6,2017-06-01,2017-06,23469.98,112.027950,20950.111111,101.256498
7,2017-07-01,2017-07,24489.17,113.759150,21527.209020,104.045739
8,2017-08-01,2017-08,24700.42,115.330667,21417.044325,103.513289
9,2017-09-01,2017-09,25136.35,117.263467,21435.789607,103.603889


### 6.4 Final integrated dataset

In [69]:
con.execute("""
CREATE OR REPLACE VIEW vw_monthly_final AS
SELECT
  c.month_date,
  c.year_month,
  c.tavg,
  c.z_tavg,
  c.climate_regime,
  c.ipc_ng_mom,
  c.ipc_food_mom,
  w.real_wage_index_base100
FROM vw_core_with_extremes c
LEFT JOIN vw_real_wage_base100 w USING (month_date, year_month)
ORDER BY c.month_date;
""")


In [70]:
con.execute("SELECT * FROM vw_monthly_final LIMIT 12").df()


,month_date,year_month,tavg,z_tavg,climate_regime,ipc_ng_mom,ipc_food_mom,real_wage_index_base100
0,2017-01-01,2017-01,26.184946,1.496477,normal,1.783333,1.283333,99.957845
1,2017-02-01,2017-02,24.659286,1.200534,normal,1.883333,1.833333,100.149900
2,2017-03-01,2017-03,21.813925,0.648600,normal,2.483333,2.233333,101.364613
3,2017-04-01,2017-04,17.732722,-0.143059,normal,2.583333,2.266667,100.410490
4,2017-05-01,2017-05,14.765860,-0.718561,normal,1.600000,1.516667,100.485183
5,2017-06-01,2017-06,12.081444,-1.239276,normal,1.133333,0.766667,101.256498
6,2017-07-01,2017-07,12.011935,-1.252759,normal,1.550000,1.000000,104.045739
7,2017-08-01,2017-08,13.981452,-0.870718,normal,1.400000,1.833333,103.513289
8,2017-09-01,2017-09,15.324889,-0.610123,normal,1.666667,1.866667,103.603889
9,2017-10-01,2017-10,18.318656,-0.029401,normal,1.616667,1.616667,104.821482


## 7. Results A — contemporaneous relationship (same month)

In [71]:
con.execute("""
SELECT
  climate_regime,
  COUNT(*) AS n_months,
  AVG(ipc_ng_mom) AS avg_ipc_ng_mom,
  AVG(ipc_food_mom) AS avg_ipc_food_mom,
  AVG(real_wage_index_base100) AS avg_real_wage_base100
FROM vw_monthly_final
GROUP BY 1
ORDER BY 1;
""").df()


,climate_regime,n_months,avg_ipc_ng_mom,avg_ipc_food_mom,avg_real_wage_base100
0,cold_extreme,6,3.419444,3.308333,88.801636
1,heat_extreme,2,4.925000,5.616667,82.332438
2,normal,88,4.777083,4.839394,86.415941


## 8. Results A (extended) — lagged effects (1 month)

We test whether the economic signals appear **one month after** a climate extreme.  
This is implemented via window functions (`LAG`) and then the same regime aggregation.


In [72]:
con.execute("""
CREATE OR REPLACE VIEW vw_monthly_lag1 AS
SELECT
  *,
  LAG(z_tavg, 1) OVER (ORDER BY month_date) AS z_tavg_lag1
FROM vw_monthly_final;
""")


In [73]:
con.execute("""
CREATE OR REPLACE VIEW vw_monthly_lag1_regime AS
SELECT
  *,
  CASE
    WHEN z_tavg_lag1 >= 1.5 THEN 'heat_extreme_lag1'
    WHEN z_tavg_lag1 <= -1.5 THEN 'cold_extreme_lag1'
    ELSE 'normal_lag1'
  END AS climate_regime_lag1
FROM vw_monthly_lag1;
""")


In [74]:
con.execute("""
SELECT
  climate_regime_lag1,
  COUNT(*) AS n_months,
  AVG(ipc_ng_mom) AS avg_ipc_ng_mom,
  AVG(ipc_food_mom) AS avg_ipc_food_mom,
  AVG(real_wage_index_base100) AS avg_real_wage_base100
FROM vw_monthly_lag1_regime
GROUP BY 1
ORDER BY 1;
""").df()


,climate_regime_lag1,n_months,avg_ipc_ng_mom,avg_ipc_food_mom,avg_real_wage_base100
0,cold_extreme_lag1,6,4.283333,4.533333,87.425793
1,heat_extreme_lag1,2,5.841667,8.316667,82.875217
2,normal_lag1,88,4.697348,4.694508,86.497412


## 9. Results B — regional heterogeneity (food inflation vs national benchmark)

We join the national climate regime to **regional IPC food inflation** and compute the deviation
from the **national average** in each regime.


In [76]:
con.execute("""
CREATE OR REPLACE VIEW vw_ipc_region_with_climate AS
SELECT
  i.month_date,
  i.year_month,
  i.region,
  i.descripcion_aperturas,
  i.vm_ipc,
  c.climate_regime
FROM fact_ipc_region_monthly i
JOIN vw_climate_regime_national c
  USING (month_date, year_month);
""")


In [77]:
con.execute("""
SELECT
  region,
  climate_regime,
  COUNT(*) AS n,
  AVG(vm_ipc) AS avg_food_inflation
FROM vw_ipc_region_with_climate
WHERE descripcion_aperturas = 'Alimentos y bebidas no alcohólicas'
GROUP BY 1,2
ORDER BY region, climate_regime;
""").df()


,region,climate_regime,n,avg_food_inflation
0,Cuyo,cold_extreme,6,3.466667
1,Cuyo,heat_extreme,2,5.600000
2,Cuyo,normal,88,4.844318
3,GBA,cold_extreme,6,3.516667
4,GBA,heat_extreme,2,5.900000
5,GBA,normal,88,4.837500
6,Noreste,cold_extreme,6,2.983333
7,Noreste,heat_extreme,2,5.200000
8,Noreste,normal,88,4.832955
9,Noroeste,cold_extreme,6,2.933333


In [78]:
con.execute("""
WITH national AS (
  SELECT
    climate_regime,
    AVG(ipc_food_mom) AS national_avg
  FROM vw_monthly_final
  GROUP BY 1
)
SELECT
  r.region,
  r.climate_regime,
  AVG(r.vm_ipc) AS regional_avg_food_inflation,
  n.national_avg,
  AVG(r.vm_ipc) - n.national_avg AS diff_vs_national
FROM vw_ipc_region_with_climate r
JOIN national n
  USING (climate_regime)
WHERE r.descripcion_aperturas = 'Alimentos y bebidas no alcohólicas'
GROUP BY 1,2,4
ORDER BY region, climate_regime;
""").df()


,region,climate_regime,regional_avg_food_inflation,national_avg,diff_vs_national
0,Cuyo,cold_extreme,3.466667,3.308333,0.158333
1,Cuyo,heat_extreme,5.600000,5.616667,-0.016667
2,Cuyo,normal,4.844318,4.839394,0.004924
3,GBA,cold_extreme,3.516667,3.308333,0.208333
4,GBA,heat_extreme,5.900000,5.616667,0.283333
5,GBA,normal,4.837500,4.839394,-0.001894
6,Noreste,cold_extreme,2.983333,3.308333,-0.325000
7,Noreste,heat_extreme,5.200000,5.616667,-0.416667
8,Noreste,normal,4.832955,4.839394,-0.006439
9,Noroeste,cold_extreme,2.933333,3.308333,-0.375000
